In [ ]:
from src.generation import one_d
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Call the one_d function
results = one_d(10)

# Convert the results to a numpy array
results_array = np.array(results)

# Plot the results
plt.plot(results_array)
plt.xlabel("Index")
plt.ylabel("Value")
plt.title("Results of one_d(10)")
plt.show()

In [ ]:
def interpolate(data: list[int], detail_level: int) -> list[float]:
    results: list[float] = []
    levels_of_detail: list[float] = [t / detail_level for t in range(detail_level)]
    for index, a in enumerate(data):
        if index == len(data) - 1:
            results.append(a)
            break
        for i in range(detail_level):
            results.append(
                (lambda a, b, t: a + (b - a) * t)(
                    a, data[index + 1], levels_of_detail[i]
                )
            )
    return results

In [ ]:
detail_level = 15

interpolated_results = interpolate(results, detail_level)
interpolated_results_array = np.array(interpolated_results)
x_axis = [t / detail_level for t in range(len(interpolated_results))]


def steps(data: list[float]) -> tuple[list[float], list[float]]:
    x_extended = []
    y_extended = []
    for i, y in enumerate(data):
        x_extended.extend([i - 0.5, i + 0.5])  # Each step spans one x-axis interval
        y_extended.extend([y, y])  # Keep the y-value constant within each interval
    return x_extended, y_extended


# Plot step-like graph
plt.step(*steps(results), where="mid", label="Original Results", linewidth=2)
plt.plot(
    x_axis, interpolated_results_array, label="Interpolated Results", linestyle="--"
)
plt.title("Original and Interpolated Results")
plt.legend()
plt.show()

In [ ]:
# def moving_average(data, diameter):
#     radius = (diameter - 1) // 2
#     smoothed_data = []
#     for i in range(len(data)):
#         start = max(0, i - radius)
#         end = min(len(data), i + radius + 1)
#         smoothed_data.append(np.mean(data[start:end]))
#     return smoothed_data


def moving_average(data, diameter):
    radius = diameter // 2
    smoothed_data = []
    length_of_data = len(data)

    smoothed_data.append(data[0])
    for i in range(1, length_of_data - 1):
        if i < radius:
            weight = (radius - i) / radius
            smoothed_data.append(
                weight * data[0] + (1 - weight) * np.mean(data[0 : i + radius + 1])
            )
        elif i >= length_of_data - radius:
            weight = (radius - (length_of_data - 1 - i)) / radius
            smoothed_data.append(
                weight * data[-1] + (1 - weight) * np.mean(data[i - radius : -1])
            )
        else:
            smoothed_data.append(np.mean(data[i - radius : i + radius + 1]))
    smoothed_data.append(data[-1])

    return smoothed_data


# Smooth the derivative using polynomial smoothing
smoothed_results = moving_average(interpolated_results, int(detail_level * 1.5))
smoothed_results_array = np.array(smoothed_results)

# Plot the results
plt.plot(x_axis, interpolated_results_array, label="Interpolated Data", alpha=0.5)
plt.plot(
    x_axis,
    smoothed_results_array,
    # linestyle="-.",
    label="Smoothed Results",
    linewidth=2,
)
plt.title("Smoothed Results")
plt.legend()
plt.show()

In [ ]:
def transform_curve(
    x_values: list[float],
    y_values: list[float],
    scalar: float = 1.0,
    magnifier: float = 1.0,
    x_offset: float = 0,
    y_offset: float = 0,
) -> tuple[list[float], list[float]]:
    if scalar != 1.0:
        y_values = [y * scalar for y in y_values]

    if magnifier != 1.0:
        mean_y = np.mean(y_values)
        y_values = [mean_y + (y - mean_y) * magnifier for y in y_values]

    if x_offset:
        x_values = [x + x_offset for x in x_values]
    if y_offset:
        y_values = [y + y_offset for y in y_values]

    return x_values, y_values

In [ ]:
def calculate_curvature_constants(data: list[float], num_segments: int) -> list[float]:
    curvature_constants = [0]
    for segment_index in range(1, num_segments):
        curvature_constants.append(
            3 * (data[segment_index + 1] - data[segment_index]) / 1
            - 3 * (data[segment_index] - data[segment_index - 1]) / 1
        )
    return curvature_constants


def solve_tridiagonal_system(
    curvature_constraints: list[float],
    num_segments: int,
) -> tuple[list[float], list[float], list[float]]:
    diagonal_terms = [1]
    off_diagonal_terms = [0]
    second_derivatives = [0]
    for segment_index in range(1, num_segments):
        diagonal_terms.append(4 - off_diagonal_terms[segment_index - 1])
        off_diagonal_terms.append(1 / diagonal_terms[segment_index])
        second_derivatives.append(
            (
                curvature_constraints[segment_index]
                - second_derivatives[segment_index - 1]
            )
            / diagonal_terms[segment_index]
        )
    diagonal_terms.append(1)  # Natural spline boundary condition
    return diagonal_terms, off_diagonal_terms, second_derivatives


def calculate_coefficients(
    second_derivatives: list[float],
    off_diagonal_terms: list[float],
    data: list[float],
    num_segments: int,
) -> tuple[list[float], list[float], list[float]]:
    cubic_coefficients = [0] * (num_segments + 1)
    for segment_index in range(num_segments - 1, -1, -1):
        cubic_coefficients[segment_index] = (
            second_derivatives[segment_index]
            - off_diagonal_terms[segment_index] * cubic_coefficients[segment_index + 1]
        )

    linear_coefficients = [
        (data[i + 1] - data[i])
        - (2 * cubic_coefficients[i] + cubic_coefficients[i + 1]) / 3
        for i in range(num_segments)
    ]
    quadratic_coefficients = [
        (cubic_coefficients[i + 1] - cubic_coefficients[i]) / 3
        for i in range(num_segments)
    ]

    return cubic_coefficients, linear_coefficients, quadratic_coefficients


def calculate_spline(
    data: list[float],
    linear_coefficients: list[float],
    cubic_coefficients: list[float],
    quadratic_coefficients: list[float],
    num_segments: int,
    samples: int,
) -> tuple[list[float], list[float]]:
    x_values = []
    y_values = []
    for segment_index in range(num_segments):
        x_interp = [
            segment_index + sample / (samples + 1) for sample in range(samples + 1)
        ]
        for x_value in x_interp:
            relative_x = x_value - segment_index
            y_value = (
                data[segment_index]
                + linear_coefficients[segment_index] * relative_x
                + cubic_coefficients[segment_index] * relative_x**2
                + quadratic_coefficients[segment_index] * relative_x**3
            )
            x_values.append(x_value)
            y_values.append(y_value)

    x_values.append(len(data) - 1)
    y_values.append(data[-1])

    return x_values, y_values


def cubic_spline(
    data: list[float],
    samples: int,
) -> tuple[list[float], list[float]]:
    num_segments = len(data) - 1

    # Step 1: Compute the second derivative constraints (alpha terms)
    curvature_constraints = calculate_curvature_constants(data, num_segments)

    # Step 2: Solve the tridiagonal system to find second derivatives
    diagonal_terms, off_diagonal_terms, second_derivatives = solve_tridiagonal_system(
        curvature_constraints, num_segments
    )

    # Step 3: Back-substitute to compute cubic coefficients
    cubic_coefficients, linear_coefficients, quadratic_coefficients = (
        calculate_coefficients(
            second_derivatives, off_diagonal_terms, data, num_segments
        )
    )

    # Step 4: Generate the interpolated x and y values

    # Step 5: Apply scalar, magnifier, and offset transformations
    x_values, y_values = calculate_spline(
        data,
        linear_coefficients,
        cubic_coefficients,
        quadratic_coefficients,
        num_segments,
        samples,
    )

    return x_values, y_values

In [ ]:
smoothed_results = cubic_spline(results, 16)
smoothed_results_array = np.array(smoothed_results)

# Plot the results
plt.plot(x_axis, interpolated_results_array, label="Interpolated Data", alpha=1)
plt.plot(
    *smoothed_results,
    label="Smoothed Results",
    linewidth=2,
)
plt.title("Smoothed Results")
plt.grid()
plt.legend()
plt.show()

In [ ]:
def extract_from_spline(values: tuple[list[float], list[float]]) -> list[float]:
    """
    Extracts y-axis values corresponding to the root values that were used to generate the spline curve.

    Args:
        values (Tuple): A tuple containing two lists of equal length that represent a spline curve.:
            values[0] (List[Float]): Corrisponds to values on the x-axis
            values[1] (List[Float]): Corrisponds to values on the y-axis

    Returns (List[Float]):
        A list of root y-axis values that were used to generate the spline curve.
    """
    return [
        values[1][values[0].index(x)]
        for x in range(int(values[0][0]), int(values[0][-1]) + 1)
    ]

In [ ]:
# Example data
data = [3, 5, 8, 4]

# Generate smoothed spline
values = cubic_spline(data, 64)

# Extract original values
original_values = extract_from_spline(values)

# Plot the results
plt.plot(*values, "-", label="Cubic Spline")  # Smoothed cubic spline
plt.plot(
    range(len(original_values)),
    original_values,
    "x",
    label="Extracted Original Values",
    markersize=8,
)  # Extracted points

plt.xlabel("X")
plt.ylabel("Y")
plt.title("Cubic Spline and Extracted Original Values")
plt.legend()
plt.grid(True)
plt.show()